In [3]:
cd ./Les_miserables/


C:\Users\berte\Desktop\M2_AIC\NLP\Projet\Les_miserables


In [4]:
ls

 Le volume dans le lecteur C s'appelle Windows
 Le num‚ro de s‚rie du volume est 28E6-BFFF

 R‚pertoire de C:\Users\berte\Desktop\M2_AIC\NLP\Projet\Les_miserables

08/11/2019  09:36    <DIR>          .
08/11/2019  09:36    <DIR>          ..
08/11/2019  09:38           480ÿ885 miserable.txt
               1 fichier(s)          480ÿ885 octets
               2 R‚p(s)  137ÿ460ÿ723ÿ712 octets libres


In [5]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import os
import nltk
# load ascii text and covert to lowercase

full_text = []

for filename in os.listdir(os.getcwd()):
    if (filename!='sample_data' and filename!='.config' and filename !='.ipynb_checkpoints'):
        raw_text = open(filename, 'r', encoding='utf-8').read()
        full_text.append(raw_text.lower())
flat_list = (', '.join(full_text))
print(len(flat_list))
flat_list = flat_list.rstrip()

Using TensorFlow backend.


462162


In [6]:
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
answer = flat_list.replace("é" , "e")
answer = answer.replace("à" , "a")
answer= answer.replace("è" , "e")
answer= answer.replace("ê" , "e")
answer = ''.join(filter(whitelist.__contains__, answer))


In [7]:
len(answer)

435632

In [8]:
nltk.download('punkt')
tokens = nltk.word_tokenize(answer)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\berte\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
print(tokens[:100])

['en', 'm', 'charlesfranoisbienvenu', 'myriel', 'etait', 'eveque', 'de', 'digne', 'cetait', 'un', 'vieillard', 'denviron', 'soixantequinze', 'ans', 'il', 'occupait', 'le', 'siege', 'de', 'digne', 'depuis', 'quoique', 'ce', 'detail', 'ne', 'touche', 'en', 'aucune', 'maniere', 'au', 'fond', 'meme', 'de', 'ce', 'que', 'nous', 'avons', 'a', 'raconter', 'il', 'nest', 'peutetre', 'pas', 'inutile', 'ne', 'ftce', 'que', 'pour', 'etre', 'exact', 'en', 'tout', 'dindiquer', 'ici', 'les', 'bruits', 'et', 'les', 'propos', 'qui', 'avaient', 'couru', 'sur', 'son', 'compte', 'au', 'moment', 'o', 'il', 'etait', 'arrive', 'dans', 'le', 'diocese', 'vrai', 'ou', 'faux', 'ce', 'quon', 'dit', 'des', 'hommes', 'tient', 'souvent', 'autant', 'de', 'place', 'dans', 'leur', 'vie', 'et', 'surtout', 'dans', 'leur', 'destinee', 'que', 'ce', 'quils', 'font', 'm']


In [10]:
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

Total Tokens: 76025
Unique Tokens: 12065


In [11]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
 


In [12]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

# save sequences to file
out_filename = 'miserable_sentences.txt'
save_doc(sequences, out_filename)

Total Sequences: 75974


In [13]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = 'miserable_sentences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

lines = lines[:100000]

In [21]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences

In [6]:
print(len(nltk.word_tokenize(lines[5])))

51


In [22]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)



In [8]:
print(sequences[5])
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

[149, 1, 80, 85, 10, 150, 703, 734, 75, 6, 602, 4, 601, 1, 80, 92, 165, 13, 445, 21, 428, 12, 409, 404, 19, 207, 56, 1, 13, 9, 44, 218, 5, 393, 6, 103, 83, 20, 358, 21, 357, 9, 22, 77, 353, 12, 31, 254, 145, 7, 278]


In [9]:
print(lines[0])

en m charlesfranoisbienvenu myriel etait eveque de digne cetait un vieillard denviron soixantequinze ans il occupait le siege de digne depuis quoique ce detail ne touche en aucune maniere au fond meme de ce que nous avons a raconter il nest peutetre pas inutile ne ftce que pour etre exact en


In [0]:
from keras.utils import to_categorical
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
 

In [11]:
import numpy as np
print(np.where(y[3]==1))
print(X[4][49])

(array([145]),)
145


In [12]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            626750    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 12535)             1266035   
Total params: 2,043,685
Trainable params: 2,043,685
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)


seq_length = len(lines[0].split()) - 1

# load the model

model = load_model("../WEIGHT_NLP2/model_fr_V4.h5")

# load the tokenizer
tokenizer = load(open("../WEIGHT_NLP2/tokenizer_fr.pkl", 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text,25)
print(generated)

dme par le diableun jour une douairiere de la variete impertinente qui se croit spirituelle lui adressa cette saillie monseigneur on demande quand votre grandeur aura le bonnet rouge oh oh voila une grosse couleur repondit leveque heureusement que ceux qui la meprisent dans un bonnet la venerent dans un chapeaui

une restrictionon risquerait fort et en allant avec une idoltrie lchant femme et lorateur nommee a terre et le pere madeleine etait devenue desole rendirent
